In [1]:
import json
import requests

In [2]:
add_ons = {}

In [37]:
nhl_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams')
nhl_dict = nhl_dict.json()

mlb_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/baseball/mlb/teams')
mlb_dict = mlb_dict.json()

nfl_dict = requests.get('http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/franchises?limit=50')
nfl_dict = nfl_dict.json()

nba_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams')
nba_dict = nba_dict.json()

#nhl_dict = requests.get('http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams')
#nhl_dict = nhl_dict.json()

In [40]:
team_links = nfl_dict['items']
for team in team_links:
    team_info = requests.get(team['$ref'])
    team_info = team_info.json()
    teamLink = requests.get(team_info['team']['$ref'])
    teamLink = teamLink.json()
    name = teamLink['displayName']
    logo = teamLink['logos'][0]['href']
    color = teamLink['color']
    alternateColor = teamLink['alternateColor']
    add_ons.update({
        name: {
            'logo': logo,
            'color': color,
            'alternateColor': alternateColor,
            'teamLink': team_info['team']['$ref']
            'venue': {
                'name': teamLink['team']['franchise']['venue']['fullName'],
                'image': teamLink['team']['franchise']['venue']['images'][0]['href'],
                'capacity': teamLink['team']['franchise']['venue']['capacity'],
                'link': teamLink['team']['franchise']['venue']['$ref']
            }
        }})
    
    


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1973168445.py, line 16)

In [6]:
nhl_dict.keys()

dict_keys(['sports'])

In [20]:
team_number = nhl_dict['sports'][0]['leagues'][0]['teams'][0]['team']['id']
team_info = requests.get(f'http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams/{team_number}')
team_info = team_info.json()

In [22]:
team_info.keys()

dict_keys(['team'])

In [35]:
team_info['team']['franchise']['venue']['capacity']

17174

In [54]:
for x in nhl_dict['sports'][0]['leagues'][0]['teams']:
    team_number = x['team']['id']
    team_link = f'http://site.api.espn.com/apis/site/v2/sports/hockey/nhl/teams/{team_number}'
    team_info = requests.get(team_link)
    team_info = team_info.json()
    
    
    name = team_info['team']['name']
    logo = team_info['team']['logos'][0]['href']
    color = team_info['team']['color']
    alternateColor = team_info['team']['alternateColor']
    teamLink = team_link
    try:
        image = team_info['team']['franchise']['venue']['images'][0]['href'],
        print(team_info['team']['franchise']['venue'])
    except:
        print(team_number)
        print(team_info['team']['franchise'])
    add_ons.update({
        name: {
            'logo': logo,
            'color': color,
            'alternateColor': alternateColor,
            'teamLink': team_link,
            'venue': {
                'name': team_info['team']['franchise']['venue']['fullName'],
                
                'capacity': team_info['team']['franchise']['venue']['capacity'],
                'link': team_info['team']['franchise']['venue']['$ref']
            }
        }})

{'$ref': 'http://sports.core.api.espn.pvt/v2/sports/hockey/leagues/nhl/venues/1842?lang=en&region=us', 'id': '1842', 'fullName': 'Honda Center', 'address': {'city': 'Anaheim', 'state': 'CA', 'zipCode': '92899', 'country': 'USA'}, 'capacity': 17174, 'grass': False, 'indoor': True, 'images': [{'href': 'https://a.espncdn.com/i/venues/nhl/day/1842.jpg', 'width': 2000, 'height': 1125, 'alt': '', 'rel': ['full', 'day']}]}
{'$ref': 'http://sports.core.api.espn.pvt/v2/sports/hockey/leagues/nhl/venues/1829?lang=en&region=us', 'id': '1829', 'fullName': 'Gila River Arena', 'address': {'city': 'Glendale', 'state': 'AZ', 'zipCode': '85318', 'country': 'USA'}, 'capacity': 17125, 'grass': False, 'indoor': True, 'images': [{'href': 'https://a.espncdn.com/i/venues/nhl/day/1829.jpg', 'width': 2000, 'height': 1125, 'alt': '', 'rel': ['full', 'day']}]}
{'$ref': 'http://sports.core.api.espn.pvt/v2/sports/hockey/leagues/nhl/venues/1824?lang=en&region=us', 'id': '1824', 'fullName': 'TD Garden', 'address': {'

In [ ]:
for x in nba_dict['sports'][0]['leagues'][0]['teams']:
    name = x['team']['displayName']
    logo = x['team']['logos'][0]['href']
    color = x['team']['color']
    alternateColor = x['team']['alternateColor']
    teamLink = ['$ref']
    add_ons.update({
        name: {
            'logo': logo,
            'color': color,
            'alternateColor': alternateColor,
            'teamLink': x['team']['$ref']
        }})

In [ ]:
for x in mlb_dict['sports'][0]['leagues'][0]['teams']:
    name = x['team']['displayName']
    logo = x['team']['logos'][0]['href']
    color = x['team']['color']
    alternateColor = x['team']['alternateColor']
    teamLink = ['$ref']
    add_ons.update({
        name: {
            'logo': logo,
            'color': color,
            'alternateColor': alternateColor,
            'teamLink': x['team']['$ref']
        }})

In [ ]:
f = open('teams.json')
data = json.load(f)
t = open('timezones.json')
timezones = json.load(t)

In [ ]:
sorted(list(data.keys()))

In [ ]:
data['MLB'][0]

In [ ]:
clean_data = {}
all_teams_info = {}
for sport in data:
    for team in data[sport]:
        teamName = team['teamName']
        city = team['city']
        state = team['state']
        metroArea = team['metroArea']
        timeZone = timezones[metroArea]
        team.update({'league':sport})
        team.update({'timeZone':timeZone})
        if sport != 'MLS':
            team.update({'logo': add_ons[teamName]['logo']})
            team.update({'color': add_ons[teamName]['color']})
            team.update({'alternateColor': add_ons[teamName]['alternateColor']})
            team.update({'teamLink': add_ons[teamName]['teamLink']})
        if metroArea not in clean_data:
            clean_data.update({metroArea: [team]})
        else:
            area = clean_data[metroArea]
            area.append(team)
            clean_data.update({metroArea: area})
        
        all_teams_info.update({teamName: team})


In [ ]:
final_data = []
number = 0
for x in clean_data:
    dict = {
        'id': number,
        'metroArea': x,
            'teams': clean_data[x]
           }
    final_data.append(dict)
    number += 1

In [ ]:
final_data = sorted(final_data, key=lambda d: d['metroArea'])

In [ ]:
json_object = json.dumps(final_data, indent=4)
with open("metro_area_teams.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
len(final_data)

In [ ]:
with open('MetroTeams.js', 'w') as f:
    f.write(f'export const TeamsCatalog = {final_data}')
    f.close()

In [ ]:
with open('AllTeamsInfo.js', 'w') as f:
    f.write(f'export const AllTeams = {all_teams_info}')
    f.close()